This program allows anyone to be able to input a protein sequence in which it will find and evaluate any EF hands the protein may have. 

In [9]:
from tensorflow import keras
model = keras.models.load_model('epoch150.h5')
maxlen = 12

In [10]:
sequence = "PTTTTKVDIAAFDPDKDGTIDLKEALAAGSAAFDKLDPDKDGTLDAKELKGRVSEADLKKLDPDNDGTLDKKEYLAAVEAQFKAANPDNDGTIDARELASPAGSALVNLIRHHHHHH" #@param {type:"string"}
aminoAcidSearch = ["D","E","A"]
efHandLength = 12

In [11]:
# Converts an EF hand into its vector form
def trans(str):
    a = []
    dic = {'A':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'H':7, 'I':8, 'K':9, 'L':10, 'M':11, 'N':12, 'P':13, 'Q':14, 'R':15, 'S':16, 'T':17, 'V':18, 'W':19, 'Y':20, 'X':21}
    for i in range(len(str)):
        a.append(dic.get(str[i]))
    return a

In [12]:
# Creates a list of potential EF hands then converts them into their vector form
position = 0
potentialEfHands = []
for i in sequence:
  i = i.upper()
  # Chooses potential EF hands by seeing if the sequence starts with a certain amino acid
  if i in aminoAcidSearch:
    # print(sequence[position:position+maxlen],trans(sequence[position:position+maxlen]))
    potentialEfHands.append([trans(sequence[position:position+maxlen]),sequence[position:position+maxlen],position+1])
  position+=1

In [13]:
# Predicts each sequence from the portentialEfHands list and adds it to the results list 
results = []
for x in potentialEfHands:
    # print(x[0])
  if len(x[0]) == 12: 
    results.append([model.predict([x[0]])[0][0],x[1],x[2]])
    # print(model.predict(x[0]))
print(results)

[[1.6322603e-05, 'DIAAFDPDKDGT', 8], [0.8581537, 'AAFDPDKDGTID', 10], [8.894166e-05, 'AFDPDKDGTIDL', 11], [0.99976265, 'DPDKDGTIDLKE', 13], [0.8176832, 'DKDGTIDLKEAL', 15], [1.8488403e-05, 'DGTIDLKEALAA', 17], [4.688988e-07, 'DLKEALAAGSAA', 21], [0.00021702051, 'EALAAGSAAFDK', 24], [0.017753541, 'ALAAGSAAFDKL', 25], [0.9920325, 'AAGSAAFDKLDP', 27], [0.993413, 'AGSAAFDKLDPD', 28], [0.38753134, 'AAFDKLDPDKDG', 31], [1.3707395e-06, 'AFDKLDPDKDGT', 32], [0.034606934, 'DKLDPDKDGTLD', 34], [0.99994725, 'DPDKDGTLDAKE', 37], [0.9996867, 'DKDGTLDAKELK', 39], [0.00053450465, 'DGTLDAKELKGR', 41], [0.006547928, 'DAKELKGRVSEA', 45], [0.0025244653, 'AKELKGRVSEAD', 46], [0.14125693, 'ELKGRVSEADLK', 48], [0.00012847781, 'EADLKKLDPDND', 55], [0.00010248512, 'ADLKKLDPDNDG', 56], [3.079093e-05, 'DLKKLDPDNDGT', 57], [0.99973553, 'DPDNDGTLDKKE', 62], [0.9997863, 'DNDGTLDKKEYL', 64], [0.0051603317, 'DGTLDKKEYLAA', 66], [1.8480496e-06, 'DKKEYLAAVEAQ', 70], [2.2098607e-06, 'EYLAAVEAQFKA', 73], [3.4212012e-06,

In [14]:
# Filters the results by checking their evaluation value
filteredResults = []
filter = 0.9
for x in results: 
  if x[0] > filter:
    print(x)
    filteredResults.append(x)
results = filteredResults

[0.99976265, 'DPDKDGTIDLKE', 13]
[0.9920325, 'AAGSAAFDKLDP', 27]
[0.993413, 'AGSAAFDKLDPD', 28]
[0.99994725, 'DPDKDGTLDAKE', 37]
[0.9996867, 'DKDGTLDAKELK', 39]
[0.99973553, 'DPDNDGTLDKKE', 62]
[0.9997863, 'DNDGTLDKKEYL', 64]
[0.95767367, 'ANPDNDGTIDAR', 85]
[0.9821944, 'DNDGTIDARELA', 88]


In [15]:
# Removes half cut EF hands
count = 0
filtered = results.copy()
for x in results: 
  best = x
  if count + 1 < len(results):
    temp = count + 1
    while results[temp][2] <= best[2] + 12:
      print(results[temp],best)

      if results[temp][1].count('D') < best[1].count('D'):
        print('removed')
        if results[temp] in filtered:
          filtered.remove(results[temp])
      elif results[temp][0] <= best[0]:
        print('removed') 
        if results[temp] in filtered:
          filtered.remove(results[temp])
      else: 
        if best in filtered: 
          filtered.remove(best)
        best = results[temp]
      temp = temp + 1
      if temp > len(results)-1:
        break
    count = count + 1
results = filtered
# Prints resulting potential and confirmed EF hands including the sequence and it's location in the protein
print(results)

[0.993413, 'AGSAAFDKLDPD', 28] [0.9920325, 'AAGSAAFDKLDP', 27]
[0.99994725, 'DPDKDGTLDAKE', 37] [0.993413, 'AGSAAFDKLDPD', 28]
[0.9996867, 'DKDGTLDAKELK', 39] [0.99994725, 'DPDKDGTLDAKE', 37]
removed
[0.99994725, 'DPDKDGTLDAKE', 37] [0.993413, 'AGSAAFDKLDPD', 28]
[0.9996867, 'DKDGTLDAKELK', 39] [0.99994725, 'DPDKDGTLDAKE', 37]
removed
[0.9996867, 'DKDGTLDAKELK', 39] [0.99994725, 'DPDKDGTLDAKE', 37]
removed
[0.9997863, 'DNDGTLDKKEYL', 64] [0.99973553, 'DPDNDGTLDKKE', 62]
removed
[0.9821944, 'DNDGTIDARELA', 88] [0.95767367, 'ANPDNDGTIDAR', 85]
[[0.99976265, 'DPDKDGTIDLKE', 13], [0.99994725, 'DPDKDGTLDAKE', 37], [0.99973553, 'DPDNDGTLDKKE', 62], [0.9821944, 'DNDGTIDARELA', 88]]


In [17]:
confirmed = 0
potential = 0
for x in results: 
  if x[0] >= 0.99:
    confirmed = confirmed + 1
  else: 
    potential = potential + 1
print("Number of confirmed ef hands: " , confirmed)
print("Nubmer of potential ef hands: ", potential)

Number of confirmed ef hands:  3
Nubmer of potential ef hands:  1
